**DataFrame Basic Operations**

In [0]:
#DataFrame is a Dataset organized into named columns.
#Distributed collection of data organized into named columns, similar to a table in a relational database or a data frame in pandas.
# DataFrames can be created from various sources (CSV, JSON, Parquet, JDBC, Hive tables, RDDs

In [0]:
#Creation of Dataframe
#1.From list of Tuples
Data=[('Jon','Doe',30),('Jan','Smith','29')]
DataFrame=spark.createDataFrame(Data,['First_Name','Last_Name','Age'])
DataFrame.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Initialize SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()

# Define the schema
schema = StructType([
    StructField("Emp_Id", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Dept", StringType(), True),
    StructField("Salary", IntegerType(), True)
])

# Read CSV file with the defined schema
dataFrame2 = spark.read.csv('dbfs:/FileStore/empdata.csv', schema=schema, header=False)

# Show DataFrame
dataFrame2.show()
dataFrame2.printSchema()
dataFrame2.count()


+------+-------+-----+------+
|Emp_Id|   Name| Dept|Salary|
+------+-------+-----+------+
|     1|   Raju|   IT| 67000|
|     2| Aditya|   HR| 59000|
|     3|  Vidya|   HR| 54000|
|     4|  Kriti|   IT| 55000|
|     5|   Ravi|   IT| 87000|
|     6|  Rahul|   IT| 83000|
|     7| Amrita|   HR| 95000|
|     8|  Kiran|Sales| 79000|
|     9|  Madhu|Sales| 30000|
|    10|  Vinay|Sales| 55000|
|    11|  Ramya|   IT| 99000|
|    12| Ambika|   HR| 45000|
|    13|  Aruna|   HR| 64000|
|    14|  Durga|   IT| 67000|
|    15| Ramesh|Sales| 78000|
|    16|  Varun|Sales| 25000|
|    17|  Vinod|Sales| 99000|
|    18|  Raghu|   HR| 54000|
|    19|Sanjeev|   IT| 58000|
|    20| Shreya|   IT| 86000|
+------+-------+-----+------+
only showing top 20 rows

root
 |-- Emp_Id: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Dept: string (nullable = true)
 |-- Salary: integer (nullable = true)

Out[2]: 25

In [0]:
#Basic Transformations
dataFrame2.filter(dataFrame2['Salary']>20000).show()

In [0]:
dataFrame2.groupBy("Dept").count().show()

In [0]:
from pyspark.sql import functions as F
result = dataFrame2.groupBy("dept").agg(F.sum("salary").alias("total_expense"))
result.show()

In [0]:
from pyspark.sql.functions import lower, col
dataFrame3=dataFrame2.filter(lower(col("Name")).startswith('r'))
dataFrame3.show()

In [0]:
from pyspark.sql import functions as F
result=dataFrame2.groupby("Dept").agg(F.round(F.avg("Salary"),2).alias("Average_Expense"))
result.show()


In [0]:
dataFrame2.orderBy(F.col("Salary").desc()).show(25)

In [0]:
dataFrame2.orderBy(F.col("Dept").asc(),F.col("Salary").desc()).show()

In [0]:
from pyspark.sql import functions as F
df_with_raise = dataFrame2.withColumn("salary_after_raise", F.round(F.col("salary") * 1.10,2))
df_with_raise.show()

In [0]:
df_with_category = dataFrame2.withColumn(
    "salary_category",
     F.when(F.col("salary") > 70000, "High")
     .when((F.col("salary") > = 30000) & (F.col("salary") <= 70000), "Medium")
     .otherwise("Low")
)

df_with_category.show()


In [0]:
dept_high=dataFrame2.groupBy('Dept').agg(F.max("Salary").alias("max_salary"))
dept_high.show()

In [0]:
df_uppercase = dataFrame2.withColumn("employee_name", F.upper(F.col("Name")))
df_uppercase.show()

In [0]:
dataFrame2.filter(dataFrame2['Salary'].isNotNull()).groupBy("Salary").count().show()

+------+-----+
|Salary|count|
+------+-----+
| 78000|    1|
| 54000|    2|
| 99000|    2|
| 30000|    1|
| 71000|    1|
| 58000|    1|
| 25000|    1|
| 95000|    1|
| 79000|    1|
| 64000|    1|
| 83000|    1|
| 47000|    1|
| 86000|    1|
| 90000|    1|
| 67000|    2|
| 55000|    2|
| 66000|    1|
| 59000|    1|
| 45000|    2|
| 87000|    1|
+------+-----+



In [0]:
spark.catalog.listTables()

Out[10]: []

In [0]:
dataFrame2.createOrReplaceTempView("users")
spark.catalog.listTables()

Out[13]: [Table(name='users', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
qry1 = "SELECT dept, count(*) as count FROM users WHERE Salary >= 50000 GROUP BY dept ORDER BY count DESC;"
sqlDF = spark.sql(qry1)
sqlDF.show()

+-----+-----+
| dept|count|
+-----+-----+
|   IT|    9|
|Sales|    6|
|   HR|    5|
+-----+-----+



In [0]:
qry2 = """
SELECT Name, Dept, Max_salary
FROM (
    SELECT Name, Dept, Salary AS Max_salary,
           Rank() OVER (PARTITION BY Dept ORDER BY Salary DESC) AS rn
    FROM users
) tmp
WHERE rn = 1
ORDER BY Max_salary DESC;
"""
sqlDF = spark.sql(qry2)
sqlDF.show()

+------+-----+----------+
|  Name| Dept|Max_salary|
+------+-----+----------+
| Ramya|   IT|     99000|
| Vinod|Sales|     99000|
|Amrita|   HR|     95000|
+------+-----+----------+



In [0]:
sqlDF.write.format("csv").option("header", "true").mode("overwrite").saveAsTable("users_table")

In [0]:
sqlDF.write.format("csv").option("header", "true").mode("overwrite").saveAsTable("users_table")